In [1]:
import tensorflow as tf
import numpy as np
from scipy.misc import imsave
import os
import shutil
from PIL import Image
import time
import random
import sys
from model import *

%matplotlib inline

In [2]:
to_train = True
to_test = False
to_restore = False
save_model = False
save_train_image = True
save_scalar = False
save_graph = False

In [3]:
class CycleGAN():

    def __init__(self):
        self.img_height = 256
        self.img_width = 256
        self.img_layer = 3
        self.batch_size = 1
        self.img_size = self.img_height * self.img_width
        
        self.pool_size = 50
        self.learning_rate = 0.0002
        self.mylambda = 10
        self.max_epoch = 20
        self.max_images = 100
        
        self.check_dir_write = "./output/write/"
        self.check_dir = "./output/checkpoints/"
        self.train_path_a = "datasets/apple2orange/trainA/*"
        self.train_path_b = "datasets/apple2orange/trainB/*"


    def input_setup(self):

        '''
        This function basically setup variables for taking image input.
        filenames_A/filenames_B -> takes the list of all training images
        self.image_A/self.image_B -> Input image with each values ranging from [-1,1]
        '''

        filenames_A = tf.train.match_filenames_once(self.train_path_a)
        print(filenames_A)
        self.queue_length_A = tf.size(filenames_A)  #图片A的数目
        print(self.queue_length_A)
        filenames_B = tf.train.match_filenames_once(self.train_path_b)
        self.queue_length_B = tf.size(filenames_B)  #图片B的数目

        filename_queue_A = tf.train.string_input_producer(filenames_A)
        filename_queue_B = tf.train.string_input_producer(filenames_B)

        image_reader = tf.WholeFileReader()
        _, image_file_A = image_reader.read(filename_queue_A)
        _, image_file_B = image_reader.read(filename_queue_B)
        
        #rgb/127.5-1映射到[-1,1]
        self.image_A = tf.subtract(tf.div(tf.image.resize_images(tf.image.decode_jpeg(image_file_A),[256,256]),127.5),1) 
        #rgb/127.5-1映射到[-1,1]
        self.image_B = tf.subtract(tf.div(tf.image.resize_images(tf.image.decode_jpeg(image_file_B),[256,256]),127.5),1)    



    def input_read(self, sess):

        '''
        It reads the input into from the image folder.
        self.fake_images_A/self.fake_images_B -> List of generated images used for calculation of loss function of Discriminator
        self.A_input/self.B_input -> Stores all the training images in python list
        '''

        # Loading images into the tensors
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord)

        num_files_A = sess.run(self.queue_length_A)
        num_files_B = sess.run(self.queue_length_B)

        self.fake_images_A = np.zeros((self.pool_size,1,self.img_height, self.img_width, self.img_layer))
        self.fake_images_B = np.zeros((self.pool_size,1,self.img_height, self.img_width, self.img_layer))


        self.A_input = np.zeros((self.max_images, self.batch_size, self.img_height, self.img_width, self.img_layer))
        self.B_input = np.zeros((self.max_images, self.batch_size, self.img_height, self.img_width, self.img_layer))

        for i in range(self.max_images):
            image_tensor = sess.run(self.image_A)

            if image_tensor.size == self.img_size*self.batch_size*self.img_layer :
                self.A_input[i] = image_tensor.reshape((self.batch_size,self.img_height, self.img_width, self.img_layer))

        for i in range(self.max_images):
            image_tensor = sess.run(self.image_B)

            if image_tensor.size == self.img_size*self.batch_size*self.img_layer :
                self.B_input[i] = image_tensor.reshape((self.batch_size,self.img_height, self.img_width, self.img_layer))


        coord.request_stop()
        coord.join(threads)




    def model_setup(self):

        ''' This function sets up the model to train
        self.input_A/self.input_B -> Set of training images.
        self.fake_A/self.fake_B -> Generated images by corresponding generator of input_A and input_B
        self.lr -> Learning rate variable
        self.cyc_A/ self.cyc_B -> Images generated after feeding self.fake_A/self.fake_B to corresponding generator. 
        This is use to calcualte cyclic loss
        '''

        self.input_A = tf.placeholder(tf.float32, [self.batch_size, self.img_width, self.img_height, self.img_layer], name="input_A")
        self.input_B = tf.placeholder(tf.float32, [self.batch_size, self.img_width, self.img_height, self.img_layer], name="input_B")

        self.fake_pool_A = tf.placeholder(tf.float32, [None, self.img_width, self.img_height, self.img_layer], name="fake_pool_A")
        self.fake_pool_B = tf.placeholder(tf.float32, [None, self.img_width, self.img_height, self.img_layer], name="fake_pool_B")

        self.global_step = tf.Variable(0, name="global_step", trainable=False)

        self.num_fake_inputs = 0

        self.lr = tf.placeholder(tf.float32, shape=[], name="lr")

        with tf.variable_scope("Model") as scope:
            self.fake_B = build_generator_resnet_9blocks(self.input_A, name="g_A")  #fake_B是由A生成的B
            self.fake_A = build_generator_resnet_9blocks(self.input_B, name="g_B")  #fake_A是由B生成的A
            self.rec_A = build_gen_discriminator(self.input_A, "d_A")
            self.rec_B = build_gen_discriminator(self.input_B, "d_B")

            scope.reuse_variables()

            self.fake_rec_A = build_gen_discriminator(self.fake_A, "d_A")
            self.fake_rec_B = build_gen_discriminator(self.fake_B, "d_B")
            self.cyc_A = build_generator_resnet_9blocks(self.fake_B, "g_B")     #cyc_A是由A生成的B再生成的A
            self.cyc_B = build_generator_resnet_9blocks(self.fake_A, "g_A")     #cyc_B是由B生成的A再生成的B

            scope.reuse_variables()

            self.fake_pool_rec_A = build_gen_discriminator(self.fake_pool_A, "d_A")
            self.fake_pool_rec_B = build_gen_discriminator(self.fake_pool_B, "d_B")

            
    def loss_cal(self):

        ''' In this function we are defining the variables for loss calcultions and traning model
        d_loss_A/d_loss_B -> loss for discriminator A/B
        g_loss_A/g_loss_B -> loss for generator A/B
        *_trainer -> Variaous trainer for above loss functions
        *_summ -> Summary variables for above loss functions'''

        cyc_loss = tf.reduce_mean(tf.abs(self.input_A-self.cyc_A)) + tf.reduce_mean(tf.abs(self.input_B-self.cyc_B))

        disc_loss_A = tf.reduce_mean(tf.squared_difference(self.fake_rec_A,1))  #fake_rec_A is G_B(input A)
        disc_loss_B = tf.reduce_mean(tf.squared_difference(self.fake_rec_B,1))  #fake_rec_B is G_A(input B)

        g_loss_A = cyc_loss*self.mylambda + disc_loss_B
        g_loss_B = cyc_loss*self.mylambda + disc_loss_A

        d_loss_A = (tf.reduce_mean(tf.square(self.fake_pool_rec_A)) + tf.reduce_mean(tf.squared_difference(self.rec_A,1)))/2.0
        d_loss_B = (tf.reduce_mean(tf.square(self.fake_pool_rec_B)) + tf.reduce_mean(tf.squared_difference(self.rec_B,1)))/2.0


        optimizer = tf.train.AdamOptimizer(self.lr, beta1=0.5)

        self.model_vars = tf.trainable_variables()

        d_A_vars = [var for var in self.model_vars if 'd_A' in var.name]
        g_A_vars = [var for var in self.model_vars if 'g_A' in var.name]
        d_B_vars = [var for var in self.model_vars if 'd_B' in var.name]
        g_B_vars = [var for var in self.model_vars if 'g_B' in var.name]

        self.d_A_trainer = optimizer.minimize(d_loss_A, var_list=d_A_vars)
        self.d_B_trainer = optimizer.minimize(d_loss_B, var_list=d_B_vars)
        self.g_A_trainer = optimizer.minimize(g_loss_A, var_list=g_A_vars)
        self.g_B_trainer = optimizer.minimize(g_loss_B, var_list=g_B_vars)

        for var in self.model_vars: print(var.name)

        #Summary variables for tensorboard
        self.g_A_loss_summ = tf.summary.scalar("g_A_loss", g_loss_A)
        self.g_B_loss_summ = tf.summary.scalar("g_B_loss", g_loss_B)
        self.d_A_loss_summ = tf.summary.scalar("d_A_loss", d_loss_A)
        self.d_B_loss_summ = tf.summary.scalar("d_B_loss", d_loss_B)

        
    def save_training_images(self, sess, epoch):
        if not os.path.exists("./output/imgs"):
            os.makedirs("./output/imgs")

        for i in range(0,10):
            fake_A_temp, fake_B_temp, cyc_A_temp, cyc_B_temp = sess.run([self.fake_A, self.fake_B, self.cyc_A, self.cyc_B],
                                                                        feed_dict={self.input_A:self.A_input[i], self.input_B:self.B_input[i]})
            #*127.5原因：原来的矩阵元素均在[-1,1]区间，+1后移动到[0,2]区间，再*127.5==255/2映射到[0,255]区间，8bit图片的RGB值
            imsave("./output/imgs/fakeA_"+ str(epoch) + "_" + str(i)+".jpg",((fake_A_temp[0]+1)*127.5).astype(np.uint8))    #F：B-->A生成的图片
            imsave("./output/imgs/fakeB_"+ str(epoch) + "_" + str(i)+".jpg",((fake_B_temp[0]+1)*127.5).astype(np.uint8))    #G：A-->B生成的图片
            imsave("./output/imgs/cycA_"+ str(epoch) + "_" + str(i)+".jpg",((cyc_A_temp[0]+1)*127.5).astype(np.uint8))      #F：G（A）-->A生成的图片
            imsave("./output/imgs/cycB_"+ str(epoch) + "_" + str(i)+".jpg",((cyc_B_temp[0]+1)*127.5).astype(np.uint8))      #G：F（B）-->B生成的图片
            imsave("./output/imgs/inputA_"+ str(epoch) + "_" + str(i)+".jpg",((self.A_input[i][0]+1)*127.5).astype(np.uint8))   #A的输入图片
            imsave("./output/imgs/inputB_"+ str(epoch) + "_" + str(i)+".jpg",((self.B_input[i][0]+1)*127.5).astype(np.uint8))   #B的输入图片

            
    def fake_image_pool(self, num_fakes, fake, fake_pool):
        ''' This function saves the generated image to corresponding pool of images.
        In starting. It keeps on feeling the pool till it is full and then randomly selects an
        already stored image and replace it with new one.'''

        if(num_fakes < self.pool_size):
            fake_pool[num_fakes] = fake
            return fake
        else :
            p = random.random()
            if p > 0.5:
                random_id = random.randint(0,self.pool_size-1)
                temp = fake_pool[random_id]
                fake_pool[random_id] = fake
                return temp
            else :
                return fake


            
    def train(self):
        ''' Training Function '''

        # Load Dataset from the dataset folder
        self.input_setup()

        #Build the network
        self.model_setup()

        #Loss function calculations
        self.loss_cal()

        # Initializing the global variables;
        init = ([tf.global_variables_initializer(), tf.local_variables_initializer()])
        saver = tf.train.Saver()

        with tf.Session() as sess:
            sess.run(init)
            #Read input to nd array
            self.input_read(sess)

            #Restore the model to run the model from last checkpoint
            if to_restore:
                chkpt_fname = tf.train.latest_checkpoint(self.check_dir)
                saver.restore(sess, chkpt_fname)

            writer = tf.summary.FileWriter(self.check_dir_write)

            if not os.path.exists(self.check_dir_write):
                os.makedirs(self.check_dir_write)
                
            if not os.path.exists(self.check_dir):
                os.makedirs(self.check_dir)

            # Training Loop
            for epoch in range(sess.run(self.global_step),self.max_epoch):
                print ("In the epoch ", epoch)
                if save_model == True :
                    saver.save(sess,os.path.join(self.check_dir,"cyclegan"),global_step=epoch)

                # Dealing with the learning rate as per the epoch number
                if(epoch < 100) :
                    curr_lr = self.learning_rate
                else:
                    curr_lr = self.learning_rate - self.learning_rate*(epoch-100)/100

                if(save_train_image):
                    self.save_training_images(sess, epoch)

                # sys.exit()

                for ptr in range(0,self.max_images):
                    print("In the iteration ",ptr)
                    print("Starting",time.time()*1000.0)

                    # Optimizing the G_A network

                    _, fake_B_temp, summary_str = sess.run([self.g_A_trainer, self.fake_B, self.g_A_loss_summ], 
                                                           feed_dict={self.input_A:self.A_input[ptr], 
                                                           self.input_B:self.B_input[ptr], 
                                                           self.lr:curr_lr})

                    if save_scalar == True :
                        writer.add_summary(summary_str, epoch*self.max_images + ptr)
                    fake_B_temp1 = self.fake_image_pool(self.num_fake_inputs, fake_B_temp, self.fake_images_B)

                    # Optimizing the D_B network
                    _, summary_str = sess.run([self.d_B_trainer, self.d_B_loss_summ], 
                                              feed_dict={self.input_A:self.A_input[ptr], 
                                              self.input_B:self.B_input[ptr], self.lr:curr_lr, 
                                              self.fake_pool_B:fake_B_temp1})
                    if save_scalar == True :
                        writer.add_summary(summary_str, epoch*self.max_images + ptr)


                    # Optimizing the G_B network
                    _, fake_A_temp, summary_str = sess.run([self.g_B_trainer, self.fake_A, self.g_B_loss_summ],
                                                           feed_dict={self.input_A:self.A_input[ptr], 
                                                           self.input_B:self.B_input[ptr], self.lr:curr_lr})

                    if save_scalar == True :
                        writer.add_summary(summary_str, epoch*self.max_images + ptr)


                    fake_A_temp1 = self.fake_image_pool(self.num_fake_inputs, fake_A_temp, self.fake_images_A)

                    # Optimizing the D_A network
                    _, summary_str = sess.run([self.d_A_trainer, self.d_A_loss_summ],
                                              feed_dict={self.input_A:self.A_input[ptr], 
                                              self.input_B:self.B_input[ptr], 
                                              self.lr:curr_lr, self.fake_pool_A:fake_A_temp1})

                    if save_scalar == True :
                        writer.add_summary(summary_str, epoch*self.max_images + ptr)

                    self.num_fake_inputs+=1

                sess.run(tf.assign(self.global_step, epoch + 1))

            if  save_graph == True :
                writer.add_graph(sess.graph)

                
    def test(self):

        ''' Testing Function'''

        print("Testing the results")

        self.input_setup()

        self.model_setup()
        saver = tf.train.Saver()
        init = tf.global_variables_initializer()

        with tf.Session() as sess:

            sess.run(init)

            self.input_read(sess)

            chkpt_fname = tf.train.latest_checkpoint(self.check_dir)
            saver.restore(sess, chkpt_fname)

            if not os.path.exists("./output/imgs/test/"):
                os.makedirs("./output/imgs/test/")

            for i in range(0,100):
                fake_A_temp, fake_B_temp = sess.run([self.fake_A, self.fake_B],
                                                    feed_dict={self.input_A:self.A_input[i], 
                                                    self.input_B:self.B_input[i]})
                imsave("./output/imgs/test/fakeB_"+str(i)+".jpg",((fake_A_temp[0]+1)*127.5).astype(np.uint8))
                imsave("./output/imgs/test/fakeA_"+str(i)+".jpg",((fake_B_temp[0]+1)*127.5).astype(np.uint8))
                imsave("./output/imgs/test/inputA_"+str(i)+".jpg",((self.A_input[i][0]+1)*127.5).astype(np.uint8))
                imsave("./output/imgs/test/inputB_"+str(i)+".jpg",((self.B_input[i][0]+1)*127.5).astype(np.uint8))



In [4]:
model = CycleGAN()
model.train()


<tf.Variable 'matching_filenames:0' shape=<unknown> dtype=string_ref>
Tensor("Size:0", shape=(), dtype=int32)


/home/fountainhead_gq/anaconda3/envs/tensorflow_p35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Model/g_A/c1/Conv/weights:0
Model/g_A/c1/Conv/biases:0
Model/g_A/c1/instance_norm/scale:0
Model/g_A/c1/instance_norm/offset:0
Model/g_A/c2/Conv/weights:0
Model/g_A/c2/Conv/biases:0
Model/g_A/c2/instance_norm/scale:0
Model/g_A/c2/instance_norm/offset:0
Model/g_A/c3/Conv/weights:0
Model/g_A/c3/Conv/biases:0
Model/g_A/c3/instance_norm/scale:0
Model/g_A/c3/instance_norm/offset:0
Model/g_A/r1/c1/Conv/weights:0
Model/g_A/r1/c1/Conv/biases:0
Model/g_A/r1/c1/instance_norm/scale:0
Model/g_A/r1/c1/instance_norm/offset:0
Model/g_A/r1/c2/Conv/weights:0
Model/g_A/r1/c2/Conv/biases:0
Model/g_A/r1/c2/instance_norm/scale:0
Model/g_A/r1/c2/instance_norm/offset:0
Model/g_A/r2/c1/Conv/weights:0
Model/g_A/r2/c1/Conv/biases:0
Model/g_A/r2/c1/instance_norm/scale:0
Model/g_A/r2/c1/instance_norm/offset:0
Model/g_A/r2/c2/Conv/weights:0
Model/g_A/r2/c2/Conv/biases:0
Model/g_A/r2/c2/instance_norm/scale:0
Model/g_A/r2/c2/instance_norm/offset:0
Model/g_A/r3/c1/Conv/weights:0
Model/g_A/r3/c1/Conv/biases:0
Model/g_A

/home/fountainhead_gq/anaconda3/envs/tensorflow_p35/lib/python3.5/site-packages/ipykernel_launcher.py:184: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/home/fountainhead_gq/anaconda3/envs/tensorflow_p35/lib/python3.5/site-packages/ipykernel_launcher.py:185: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/home/fountainhead_gq/anaconda3/envs/tensorflow_p35/lib/python3.5/site-packages/ipykernel_launcher.py:186: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/home/fountainhead_gq/anaconda3/envs/tensorflow_p35/lib/python3.5/site-packages/ipykernel_launcher.py:187: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite

In the iteration  0
Starting 1528268350478.3418
In the iteration  1
Starting 1528268354207.412
In the iteration  2
Starting 1528268355157.767
In the iteration  3
Starting 1528268356107.7812
In the iteration  4
Starting 1528268357057.1628
In the iteration  5
Starting 1528268358010.195
In the iteration  6
Starting 1528268358959.2886
In the iteration  7
Starting 1528268359911.373
In the iteration  8
Starting 1528268360861.2166
In the iteration  9
Starting 1528268361810.4514
In the iteration  10
Starting 1528268362760.8557
In the iteration  11
Starting 1528268363710.6648
In the iteration  12
Starting 1528268364661.1753
In the iteration  13
Starting 1528268365610.247
In the iteration  14
Starting 1528268366561.9062
In the iteration  15
Starting 1528268367513.555
In the iteration  16
Starting 1528268368463.1333
In the iteration  17
Starting 1528268369415.1816
In the iteration  18
Starting 1528268370367.033
In the iteration  19
Starting 1528268371317.892
In the iteration  20
Starting 15282683

...

In the iteration  11
Starting 1528269341899.1436
In the iteration  12
Starting 1528269342851.929
In the iteration  13
Starting 1528269343806.8164
In the iteration  14
Starting 1528269344761.7883
In the iteration  15
Starting 1528269345717.0862
In the iteration  16
Starting 1528269346671.1921
In the iteration  17
Starting 1528269347629.1353
In the iteration  18
Starting 1528269348585.6357
In the iteration  19
Starting 1528269349545.352
In the iteration  20
Starting 1528269350503.75
In the iteration  21
Starting 1528269351459.572
In the iteration  22
Starting 1528269352415.8643
In the iteration  23
Starting 1528269353371.7073
In the iteration  24
Starting 1528269354330.153
In the iteration  25
Starting 1528269355284.9893
In the iteration  26
Starting 1528269356242.491
In the iteration  27
Starting 1528269357197.0063
In the iteration  28
Starting 1528269358153.3728
In the iteration  29
Starting 1528269359110.06
In the iteration  30
Starting 1528269360065.6782
In the iteration  31
Starting

In the iteration  79
Starting 1528269504852.4004
In the iteration  80
Starting 1528269505809.2285
In the iteration  81
Starting 1528269506763.988
In the iteration  82
Starting 1528269507717.0742
In the iteration  83
Starting 1528269508673.062
In the iteration  84
Starting 1528269509626.7266
In the iteration  85
Starting 1528269510586.5237
In the iteration  86
Starting 1528269511539.7402
In the iteration  87
Starting 1528269512498.1814
In the iteration  88
Starting 1528269513451.8147
In the iteration  89
Starting 1528269514409.043
In the iteration  90
Starting 1528269515363.2173
In the iteration  91
Starting 1528269516313.6206
In the iteration  92
Starting 1528269517268.4304
In the iteration  93
Starting 1528269518220.0457
In the iteration  94
Starting 1528269519173.2224
In the iteration  95
Starting 1528269520129.4138
In the iteration  96
Starting 1528269521083.08
In the iteration  97
Starting 1528269522039.32
In the iteration  98
Starting 1528269522993.9626
In the iteration  99
Starti

In the iteration  47
Starting 1528269670044.1008
In the iteration  48
Starting 1528269670996.7302
In the iteration  49
Starting 1528269671947.9722
In the iteration  50
Starting 1528269672900.8665
In the iteration  51
Starting 1528269673854.6702
In the iteration  52
Starting 1528269674808.4631
In the iteration  53
Starting 1528269675763.4502
In the iteration  54
Starting 1528269676719.6265
In the iteration  55
Starting 1528269677672.0059
In the iteration  56
Starting 1528269678627.986
In the iteration  57
Starting 1528269679583.2942
In the iteration  58
Starting 1528269680536.821
In the iteration  59
Starting 1528269681489.657
In the iteration  60
Starting 1528269682444.4036
In the iteration  61
Starting 1528269683400.9128
In the iteration  62
Starting 1528269684353.9702
In the iteration  63
Starting 1528269685309.3132
In the iteration  64
Starting 1528269686268.1523
In the iteration  65
Starting 1528269687220.841
In the iteration  66
Starting 1528269688178.4478
In the iteration  67
Sta

In the iteration  15
Starting 1528269835212.9463
In the iteration  16
Starting 1528269836168.575
In the iteration  17
Starting 1528269837122.1462
In the iteration  18
Starting 1528269838081.486
In the iteration  19
Starting 1528269839038.1826
In the iteration  76
Starting 1528269991301.2114
In the iteration  77
Starting 1528269992254.9387
In the iteration  78
Starting 1528269993212.5195
In the iteration  79
Starting 1528269994165.3037
In the iteration  80
Starting 1528269995121.5322
In the iteration  81
Starting 1528269996075.0342
In the iteration  82
Starting 1528269997026.69
In the iteration  83
Starting 1528269997978.3342
In the iteration  84
Starting 1528269998934.236
In the iteration  85
Starting 1528269999887.4294
In the iteration  86
Starting 1528270000845.779
In the iteration  87
Starting 1528270001799.3674
In the iteration  88
Starting 1528270002752.8193
In the iteration  89
Starting 1528270003705.464
In the iteration  90
Starting 1528270004659.4822
In the iteration  91
Starti

In the iteration  39
Starting 1528270151642.5764
In the iteration  40
Starting 1528270152595.2908
In the iteration  41
Starting 1528270153552.376
In the iteration  42
Starting 1528270154507.2576
In the iteration  43
Starting 1528270155464.9617
In the iteration  44
Starting 1528270156419.1436
In the iteration  45
Starting 1528270157375.5374
In the iteration  46
Starting 1528270158331.5706
In the iteration  47
Starting 1528270159287.3176
In the iteration  48
Starting 1528270160241.421
In the iteration  49
Starting 1528270161196.2644
In the iteration  50
Starting 1528270162151.447
In the iteration  51
Starting 1528270163107.3748
In the iteration  52
Starting 1528270164059.415
In the iteration  53
Starting 1528270165017.8655
In the iteration  54
Starting 1528270165971.8513
In the iteration  55
Starting 1528270166925.665
In the iteration  56
Starting 1528270167883.9785
In the iteration  57
Starting 1528270168839.2302
In the iteration  58
Starting 1528270169791.4888
In the iteration  59
Star